In [1]:
#import the necessary libs

from pyspark.sql import SparkSession
import getpass

username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.suffle.useOldFetchProtocol",'true'). \
    config("spark.dql.warehouse.dir",f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [2]:

loans_def_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/user/itv014478/lendingclubproject/raw/loan_defaulters_data_csv")

In [3]:
loans_def_raw_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
6d5091b3fcaaeb4ea...,0.0,0.0,0.0,0.0,1.0,0.0,30.0,null
b5e7938b0a2da4cea...,1.0,0.0,0.0,0.0,4.0,0.0,6.0,null
91060b858433e8a61...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
cab1fa9f533688b0a...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
f74e401c1ab0adf78...,1.0,0.0,0.0,0.0,3.0,0.0,12.0,null
8aef4bb29d609d8d6...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
538b4653da3b1e814...,0.0,0.0,0.0,0.0,0.0,0.0,49.0,null
b24d55f21390533c5...,1.0,0.0,0.0,0.0,0.0,0.0,3.0,null
1035c5401b0ca76d0...,0.0,0.0,1.0,1.0,1.0,0.0,null,106.0
cb0f1777593e77909...,0.0,0.0,0.0,0.0,0.0,0.0,75.0,null


In [4]:
loans_def_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- total_rec_late_fee: double (nullable = true)
 |-- mths_since_last_delinq: double (nullable = true)
 |-- mths_since_last_record: double (nullable = true)



In [5]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [6]:
spark.sql("select distinct(delinq_2yrs) from loan_defaulters")

delinq_2yrs
8.0
0.0
7.0
null
18.0
1.0
39.0
10.58
4.0
11.0


In [7]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)

+-----------+------+
|delinq_2yrs| total|
+-----------+------+
|        0.0|369488|
|        1.0| 61190|
|        2.0| 18260|
|        3.0|  6646|
|        4.0|  3104|
|        5.0|  1552|
|        6.0|   885|
|        7.0|   460|
|        8.0|   299|
|        9.0|   185|
|       10.0|   139|
|       11.0|    87|
|       12.0|    54|
|       13.0|    41|
|       14.0|    39|
|       15.0|    22|
|       16.0|    12|
|       17.0|     9|
|       19.0|     6|
|       18.0|     5|
|       20.0|     2|
|       null|     2|
|       26.0|     2|
|       39.0|     1|
|      10.58|     1|
|       22.0|     1|
|       27.0|     1|
|       30.0|     1|
+-----------+------+



In [8]:
loan_defaulters_schema = "member_id string, delinq_2yrs float, delinq_amnt float, pub_rec float, pub_rec_bankruptcies float,inq_last_6mths float, total_rec_late_fee float, mths_since_last_delinq float, mths_since_last_record float"

In [9]:
loans_def_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loan_defaulters_schema) \
.load("/user/itv014478/lendingclubproject/raw/loan_defaulters_data_csv")

In [10]:
loans_def_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: float (nullable = true)
 |-- delinq_amnt: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- pub_rec_bankruptcies: float (nullable = true)
 |-- inq_last_6mths: float (nullable = true)
 |-- total_rec_late_fee: float (nullable = true)
 |-- mths_since_last_delinq: float (nullable = true)
 |-- mths_since_last_record: float (nullable = true)



In [11]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [12]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)

+-----------+------+
|delinq_2yrs| total|
+-----------+------+
|        0.0|369488|
|        1.0| 61190|
|        2.0| 18260|
|        3.0|  6646|
|        4.0|  3104|
|        5.0|  1552|
|        6.0|   885|
|        7.0|   460|
|        8.0|   299|
|        9.0|   185|
|       10.0|   139|
|       11.0|    87|
|       12.0|    54|
|       13.0|    41|
|       14.0|    39|
|       15.0|    22|
|       16.0|    12|
|       17.0|     9|
|       19.0|     6|
|       18.0|     5|
|       20.0|     2|
|       null|     2|
|       26.0|     2|
|       30.0|     1|
|      10.58|     1|
|       39.0|     1|
|       22.0|     1|
|       27.0|     1|
+-----------+------+



In [13]:
from pyspark.sql.functions import col

In [14]:
loans_def_processed_df = loans_def_raw_df.withColumn("delinq_2yrs", col("delinq_2yrs").cast("integer")).fillna(0, subset = ["delinq_2yrs"])

In [15]:
loans_def_processed_df.createOrReplaceTempView("loan_defaulters")

In [16]:
spark.sql("select count(*) from loan_defaulters where delinq_2yrs is null")

count(1)
0


In [18]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc")

delinq_2yrs,total
0,369490
1,61190
2,18260
3,6646
4,3104
5,1552
6,885
7,460
8,299
9,185


In [19]:
loans_def_delinq_df = spark.sql("select member_id,delinq_2yrs, delinq_amnt, int(mths_since_last_delinq) from loan_defaulters where delinq_2yrs > 0 or mths_since_last_delinq > 0")

In [20]:
loans_def_delinq_df

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
6d5091b3fcaaeb4ea...,0,0.0,30
b5e7938b0a2da4cea...,1,0.0,6
f74e401c1ab0adf78...,1,0.0,12
538b4653da3b1e814...,0,0.0,49
b24d55f21390533c5...,1,0.0,3
cb0f1777593e77909...,0,0.0,75
a962f4d59caec5fa1...,0,0.0,54
23857480ccf555ce4...,0,0.0,42
f2c4010f700d8c9c4...,0,0.0,29
0d3c568ff6944b11c...,0,0.0,33


In [21]:
loans_def_delinq_df.count()

236775

In [22]:
loans_def_records_enq_df = spark.sql("select member_id from loan_defaulters where pub_rec > 0.0 or pub_rec_bankruptcies > 0.0 or inq_last_6mths > 0.0")

In [23]:
loans_def_records_enq_df

member_id
6d5091b3fcaaeb4ea...
b5e7938b0a2da4cea...
f74e401c1ab0adf78...
1035c5401b0ca76d0...
9fe2d59ddf2a4f37e...
08bf9e080503b0113...
05ad4aed7c393035e...
0d3c568ff6944b11c...
18889919d4248ff48...
159b209fd25dff55b...


In [24]:
loans_def_records_enq_df.count()

219605

In [25]:
loans_def_delinq_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv014478/lendingclubproject/cleaned/loans_defaulters_deling_csv") \
.save()

In [26]:
loans_def_delinq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv014478/lendingclubproject/cleaned/loans_defaulters_deling_parquet") \
.save()

In [27]:
loans_def_records_enq_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv014478/lendingclubproject/cleaned/loans_defaulters_records_enq_csv") \
.save()

In [28]:
loans_def_records_enq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv014478/lendingclubproject/cleaned/loans_defaulters_records_enq_parquet") \
.save()